# Some Gists  

Gists are code snippets

In [ ]:
# import our modules
import requests
from leafmap import Map
from pystac_client import Client

# draw a map for reference
m = Map(
    center=(38.04855184, -84.50059457), 
    zoom=12
    )

m.add_tile_layer(
    url="https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_TCM_Base_WGS84WM/MapServer/tile/{z}/{y}/{x}",
    name="TCM",
    attribution="DGI",
)

# declare some basic variables

bbox = '-84.57479592,38.00492759,-84.41622051,38.07550761'
stac_url = 'https://spved5ihrl.execute-api.us-west-2.amazonaws.com/'
collection = 'orthos-phase3'

# initial the stac client
client = Client.open(stac_url)

# set up the search
search = client.search(
    collections=collection,
    bbox = bbox,
    limit = 100
)

# get the items
items = search.get_all_items()

cogs = []
for item in items:
    for band, asset in item.assets.items():
        if asset.media_type == "image/tiff; application=geotiff; profile=cloud-optimized":
            cogs.append(asset.href)

print(f"Found {len(cogs)} COGs")
for c in cogs[:5]:
    print(c)

Map(center=[38.04855184, -84.50059457], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_ti…

# Build a VRT  

In [ ]:
from osgeo import gdal
import leafmap

# ---- Step 1: Get your list of COG URLs ----
cogs = [asset.href for item in items for asset in item.assets.values()
        if asset.media_type == "image/tiff; application=geotiff; profile=cloud-optimized"]


# ---- Step 2: Create a VRT (virtual raster mosaic) ----
# Band order: 4 (NIR), 1 (Blue), 2 (Green)
# GDAL uses 1-based indexing for bands when using separate sources, but
# for multiple files, BuildVRT keeps original bands. We'll reorder later when reading.

vrt_path = "mosaic_4_1_2.vrt"
vrt_options = gdal.BuildVRTOptions(resolution='highest')  # 'average' or 'highest'

vrt = gdal.BuildVRT(
    vrt_path,
    cogs,
    options=vrt_options
)

vrt = None  # flush to disk
print(f"VRT saved to {vrt_path}")


# ---- Step 3: Display in Leafmap via Titiler ----
# If you have local Titiler running on localhost:8000
titiler_endpoint = "http://localhost:8000"

# Build a URL template for the VRT (Titiler can serve local files)
tile_url = f"{titiler_endpoint}/cog/tiles/{{z}}/{{x}}/{{y}}.png?bidx=4,1,2&rescale=0,3000"

m = leafmap.Map(center=[37.5, -85.5], zoom=7)
m.add_tile_layer(
    url=tile_url,
    name="Impervious Surface Mosaic (4-1-2)",
    attribution="Local Titiler"
)

m